In [3]:
from pathlib import Path
import numpy as np

import info9

In [4]:
train_ds = Path("csv/mail_train.csv")
test_ds = Path("csv/mail_test.csv") 

In [5]:
train_ds = Path("csv/mail_train.csv")
test_ds = Path("csv/mail_test.csv") 

In [6]:
d = info9.Dataset(str(train_ds))
d.show(False)

In [7]:
k = 3
classifier = info9.KnnClassification(k, d, 0)

In [8]:
a = 0
with test_ds.open("r") as f:
    a = f.readline().split(",")
len(a)

1900

In [9]:
b = np.array(a[1:], dtype=np.float64)
type(b),b.shape,b.dtype

(numpy.ndarray, (1899,), dtype('float64'))

In [ ]:
classifier.estimate(b, 0.5)